# **Import Modules**

## *python*

In [1]:
# Import python modules
import os,sys

## *custom*

In [2]:
# Import customised modules

# Define path
sys.path.append(os.path.abspath(os.path.join('..')))

##import relevant modules
try:
    from scripts._02_2_embed_and_index import EmbeddingIndexer

except ImportError:
    print('Could not import module from `scripts/`.')

# **Embedding and Indexing**

In [3]:
# Define DataFrame path
df_chunks_path = ('../data/processed/chunked_complaints.csv')
vector_store_dir = ('../vector store')

In [4]:
# Initalise class
embedder_and_indexer = EmbeddingIndexer (df_chunks_path = df_chunks_path, vector_store_dir = vector_store_dir)

Using model: all-MiniLM-L6-v2

Embedding dimension: 384


In [5]:
# Load data
embedder_and_indexer.load_chunks()


DataFrame loaded successfully from ..\data\processed\chunked_complaints.csv


,Chunk,Chunk Length,Company,Company Public Response,Company Response To Consumer,Complaint Id,Complaint Length,Consumer Complaint Narrative,Consumer Complaint Narrative Clean,Consumer Consent Provided?,...,Date Sent To Company,Issue,Product,State,Sub-Issue,Sub-Product,Submitted Via,Tags,Timely Response?,Zip Code
0,xxxx xxxx card opened name fraudster. received...,311,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with non-monetary relief,14069121,488.0,A XXXX XXXX card was opened under my name by a...,xxxx xxxx card opened name fraudster. received...,Consent provided,...,2025-06-13,Getting a credit card,Credit card,TX,Card opened without my consent or knowledge,Store credit card,Web,Servicemember,Yes,78230
1,made mistake using wellsfargo debit card depsi...,354,WELLS FARGO & COMPANY,Company has responded to the consumer and the ...,Closed with explanation,14061897,555.0,I made the mistake of using my wellsfargo debi...,made mistake using wellsfargo debit card depsi...,Consent provided,...,2025-06-13,Managing an account,Checking or savings account,ID,Deposits and withdrawals,Checking account,Web,Unknown,Yes,83815
2,secured credit card citibank changed unsecured...,496,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with monetary relief,14047085,806.0,"Dear CFPB, I have a secured credit card with c...",secured credit card citibank changed unsecured...,Consent provided,...,2025-06-13,"Other features, terms, or problems",Credit card,NY,Other problem,General-purpose credit card or charge card,Web,Unknown,Yes,11220
3,citi reward cards. credit balance issued $8400...,443,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with explanation,14040217,1199.0,I have a Citi rewards cards. The credit balanc...,citi reward cards. credit balance issued $8400...,Consent provided,...,2025-06-12,Incorrect information on your report,Credit card,IL,Account information incorrect,General-purpose credit card or charge card,Web,Unknown,Yes,60067
4,.00 $5400.00 sends message reached credit limi...,364,"CITIBANK, N.A.",Company has responded to the consumer and the ...,Closed with explanation,14040217,1199.0,I have a Citi rewards cards. The credit balanc...,citi reward cards. credit balance issued $8400...,Consent provided,...,2025-06-12,Incorrect information on your report,Credit card,IL,Account information incorrect,General-purpose credit card or charge card,Web,Unknown,Yes,60067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022259,". resolution outside court would preferred, ma...",329,WELLS FARGO & COMPANY,Company has responded to the consumer and the ...,Closed with explanation,6238646,3462.0,"I already have a case XXXX, but it should not ...","already case xxxx, closed. additional informat...",Consent provided,...,2022-11-23,Managing an account,Checking or savings account,OK,Deposits and withdrawals,Savings account,Web,Unknown,Yes,73160
1022260,xxxxxxxx final payment supposed submitted want...,214,"Westlake Services, LLC",Company has responded to the consumer and the ...,Closed with explanation,6238123,2284.0,On XX/XX/XXXX my final payment was supposed to...,xxxxxxxx final payment supposed submitted want...,Consent provided,...,2022-11-23,Vehicle was repossessed or sold the vehicle,"Payday loan, title loan, or personal loan",CA,Unknown,Title loan,Web,Older American,Yes,90059
1022261,. called immediately gave represented new addr...,429,"Westlake Services, LLC",Company has responded to the consumer and the ...,Closed with explanation,6238123,2284.0,On XX/XX/XXXX my final payment was supposed to...,xxxxxxxx final payment supposed submitted want...,Consent provided,...,2022-11-23,Vehicle was repossessed or sold the vehicle,"Payday loan, title loan, or personal loan",CA,Unknown,Title loan,Web,Older American,Yes,90059
1022262,.00 going balance rest going interest fee see ...,493,"Westlake Services, LLC",Company has responded to the consu

In [6]:
# Embed and Index chunks
embedder_and_indexer.index_chunks()

Skipping indexing: collection already contains 1022264 chunks.

Vector store saved to: ..\vector store


In [7]:
# Query ChromaDB with optional metadata filtering
query_product_pairs = [
    # Credit Cards
    ("unauthorized charges on my card", "Credit card or prepaid card", "Credit Cards"),
    ("interest rate too high", "Credit card", "Credit Cards"),

    # Personal Loans
    ("loan repayment issues", "Payday loan, title loan, or personal loan", "Personal Loans"),
    ("unexpected fees on personal loan", "Payday loan, title loan, personal loan, or advance loan", "Personal Loans"),

    # Buy Now Pay Later (BNPL) (grouped from multiple dataset labels)
    ("confusion about repayment schedule", "Payday loan", "BNPL"),
    ("unexpected late fees", "Payday loan, title loan, or personal loan", "BNPL"),
    ("loan terms not clearly explained", "Payday loan, title loan, personal loan, or advance loan", "BNPL"),
    ("hidden charges in installment plan", "Other financial service", "BNPL"),

    # Savings Accounts
    ("account frozen without notice", "Checking or savings account", "Savings Accounts"),
    ("unauthorized withdrawal from savings", "Checking or savings account", "Savings Accounts"),

    # Money Transfers
    ("money sent to wrong recipient", "Money transfers", "Money Transfers"),
    ("transfer failed but money deducted", "Money transfer, virtual currency, or money service", "Money Transfers")
]

grouped_results = embedder_and_indexer.run_batch_queries_grouped(query_product_pairs)


Query: 'unauthorized charges on my card' | Dataset Product: 'Credit card or prepaid card' | Business Product: 'Credit Cards'


,Chunk ID,Distance,Company,Issue,Chunk Text
0,chunk-329948,0.343202,"Block, Inc.",Problem with a purchase or transfer,unauthorized transaction made card
1,chunk-821640,0.375521,NAVY FEDERAL CREDIT UNION,Getting a credit card,unauthorized use credit card
2,chunk-462591,0.438429,KEYCORP,Problem with a purchase or transfer,found unauthorized transaction card



Query: 'interest rate too high' | Dataset Product: 'Credit card' | Business Product: 'Credit Cards'


,Chunk ID,Distance,Company,Issue,Chunk Text
0,chunk-290812,0.131167,DISCOVER BANK,Fees or interest,interest rate high
1,chunk-194889,0.598893,CAPITAL ONE FINANCIAL CORPORATION,Fees or interest,interest rate high. want help fix all. $110.00...
2,chunk-97573,0.652960,CAPITAL ONE FINANCIAL CORPORATION,Fees or interest,asked assistance interest rate high



Query: 'loan repayment issues' | Dataset Product: 'Payday loan, title loan, or personal loan' | Business Product: 'Personal Loans'


...

,Chunk ID,Distance,Company,Issue,Chunk Text
0,chunk-140318,0.329786,TD BANK US HOLDING COMPANY,Problem with a lender or other company chargin...,money unauthorized withdrawal account
1,chunk-11804,0.575189,GOLDMAN SACHS BANK USA,Managing an account,". unfortunately, successful withdrawing $5600...."
2,chunk-564794,0.581799,WELLS FARGO & COMPANY,Managing an account,59 unauthorized withdraws checking account aut...



Query: 'money sent to wrong recipient' | Dataset Product: 'Money transfers' | Business Product: 'Money Transfers'


,Chunk ID,Distance,Company,Issue,Chunk Text
0,chunk-738146,0.596299,"Garden State Check Cashing Service, Inc.",Fraud or scam,transferred money xxxx never received money re...
1,chunk-874198,0.643510,MONEYGRAM PAYMENT SYSTEMS WORLDWIDE INC,Money was not available when promised,sent money scam need money back
2,chunk-741817,0.656610,MONEYGRAM PAYMENT SYSTEMS WORLDWIDE INC,Other transaction issues,went cv send money via moneygram payment syste...



Query: 'transfer failed but money deducted' | Dataset Product: 'Money transfer, virtual currency, or money service' | Business Product: 'Money Transfers'


,Chunk ID,Distance,Company,Issue,Chunk Text
0,chunk-852773,0.528723,JPMORGAN CHASE & CO.,Confusing or missing disclosures,xxxx transfer went wrong place unable get mone...
1,chunk-363264,0.565152,REGIONS FINANCIAL CORPORATION,Unexpected or other fees,. deducted $39.00 return fee money never left ...
2,chunk-353948,0.575292,Sigue Corp.,Other transaction problem,"transferred $2500.00 xxxx xxxxyear, failed rec..."
